In [1]:
from helper_parthi2929 import *


romania_location_map = {
    'A' : {'pos': (21.31227,46.18656), 'connections': ['S','T','Z'] },
    'S' : {'pos': (24.12558,45.79833), 'connections': ['F','RV','O'] },
    'Z' : {'pos': (21.51742,46.62251), 'connections': ['O'] },
    'T' : {'pos': (21.20868,45.74887), 'connections': ['LU'] },
    'F' : {'pos': (24.97310,45.84164), 'connections': ['B'] },
    'LU' : {'pos': (21.90346,45.69099), 'connections': ['M'] },
    'RV' : {'pos': (24.36932,45.09968), 'connections': ['C','P'] },
    'M' : {'pos': (22.36452,44.90411), 'connections': ['D'] },
    'D' : {'pos': (22.65973,44.63692), 'connections': ['C'] },
    'P' : {'pos': (24.86918,44.85648), 'connections': ['B','C'] },
    'B' : {'pos': (26.10254,44.42677), 'connections': ['G','U'] },
    'U' : {'pos': (26.64112,44.71653), 'connections': ['H','V'] },
    'V' : {'pos': (27.72765,46.64069), 'connections': ['LA'] },
    'LA' : {'pos':(27.60144,47.15845), 'connections': ['N'] },
    'H' : {'pos': (27.94566,44.68935), 'connections': ['E'] },
    
    #'C' and 'O' are not really dead ended, but connection not included solely caz already included earlier
    # current algo starting here may have problem
    'C' : {'pos': (23.79488,44.33018), 'connections': [] },    
    'O' : {'pos': (21.91894,47.04650), 'connections': [] },        
    
    # these guys are dead ended but you can go backwards
    # currently algo starting here may have problem
    'N' : {'pos': (26.38188,46.97587), 'connections': [] },    
    'G' : {'pos': (25.96993,43.90371), 'connections': [] },    
    'E' : {'pos': (28.65273,44.04911), 'connections': [] }
}

graph = load_map_graph(romania_location_map)
#show_map(graph, start='A',goal='B',path=['A','S','F','B'],SLDTarget='B')
show_map(graph)

Let us try to find for A to U. Obviously path should is clear for us looking at map but we get a costlier path :(

In [2]:
show_map(graph, start='A',goal='U',path=['A','S','RV','P','B','U'],SLDTarget='U')

In [3]:
from queue import PriorityQueue

# calculating geodesic SLDs
import geopy.distance
def calculate_GD(start, end):

    (x0,y0) = graph.node[start]['pos']
    (x1,y1) = graph.node[end]['pos']
    return floor(geopy.distance.geodesic((y0,x0), (y1,x1)).miles)

def calculate_totalGD(node):
    cost = 0
    if cameFrom.get(node) is None: return 0
    while True:   #when current_node is not None..        
        cost += calculate_GD(cameFrom.get(node),node)
        node = cameFrom.get(node)
        if cameFrom.get(node) is None: break
    return cost

def calculate_path(node):
    path = [node]
    while node is not None:   #when current_node is not None..        
        node = cameFrom.get(node)
        path.append(node)
    return [x for x in reversed(path[:-1])]


q = PriorityQueue()
start = 'A'
goal = 'U'

Let us initialize as usual...
```
q = {(0, 'A')}  
closedSet = ()  
```

In [4]:
q.put((0,start))
closedSet = set()  # set() would be better than a list?
cameFrom = {}           
cameFrom[start] = None 

**ITERATION 1** 
<pre>
Is q empty? No. So Go on.
Take least cost path from q
{<font color="blue"><b>(0, 'A')</b></font>} : <font color="blue"><b>(0, 'A')</b></font>
Current node: 'A'. Kids: 'S','T','Z'
Is 'A' the goal? No. So Go on.  

                            closedSet : ( 'A' ) 
                            
**'A' to 'S':**          
            past cost from 'A' to 'S' : 138  
       heuristic cost from 'S' to 'U' : 143  
                                total : **281**                                
                            closedSet : ( 'A', 'S' )
**'A' to 'T':**                                         
            past cost from 'A' to 'T' : 30  
       heuristic cost from 'T' to 'U' : 274  
                                total : **304**     
                            closedSet : ( 'A', 'S', 'T' )                                
**'A' to 'Z':**                                        
            past cost from 'A' to 'Z' : 31  
       heuristic cost from 'Z' to 'O' : 280  
                                total : **311**                                   
                            closedSet : ( 'A', 'S', 'T', 'Z' )                                
                                
**Result: q = { **<font color="blue"><b>(281,'S'), (304,'T'), (311,'Z') }</b></font>
</pre>
<hr>

In [5]:
if not q.empty():
    
    current_cost, current_node = q.get()   
    
    if not (current_node == goal):
        closedSet.add(current_node)
        for each_child in romania_location_map.get(current_node,[]).get('connections',[]):
            if not each_child in closedSet:   #to avoid 'C' need not revisit
                cameFrom[each_child] = current_node #this is needed before hand to help totalGD
                past_cost = calculate_totalGD(each_child)
                heuristic_cost = calculate_GD(each_child, goal)
                total_cost = past_cost + heuristic_cost
                q.put( (total_cost, each_child) )
                closedSet.add(each_child)
#test
print(list(q.queue))
print(list(closedSet))

[(281, 'S'), (304, 'T'), (311, 'Z')]
['Z', 'T', 'S', 'A']


**ITERATION 2**  
<pre>
Is q empty? No. So Go on.
Take least cost path from q 
{ <font color="blue"><b>(281, 'S')</b></font>, (304, 'T'), (311, 'Z') } : <font color="blue"><b>(281,'S')</b></font>
Current node: 'S'. Kids: 'F','RV','O'  
Is 'S' the goal? No. So Go on.

                            closedSet : ( 'A', 'S', 'T', 'Z' )
                            
**'S' to 'F':**          
            past cost from 'A' to 'F' : 138+41
       heuristic cost from 'F' to 'U' : 112  
                                total : **291**                                  
                            closedSet : ( 'A', 'S', 'T', 'Z', 'F' )   

**'S' to 'RV':**          
            past cost from 'A' to 'RV' : 138+49
       heuristic cost from 'RV' to 'U' : 114  
                                total : **301**                                  
                            closedSet : ( 'A', 'S', 'T', 'Z', 'F', 'RV' )

**'S' to 'O':**          
            past cost from 'A' to 'O' : 138+136  (not 31+34. why?)
       heuristic cost from 'O' to 'U' : 278  
                                total : **552**                                  
                            closedSet : ( 'A', 'S', 'T', 'Z', 'F', 'RV', 'O' )
   
                            
                            
**Result: q = { (304, 'T'), (311, 'Z'), **<font color="blue"><b>(291, 'F'), (301, 'RV'), (552, 'O') }</b></font>
</pre>
<hr>

In [6]:
if not q.empty():
    
    current_cost, current_node = q.get()   
    
    if not (current_node == goal):
        closedSet.add(current_node)
        for each_child in romania_location_map.get(current_node,[]).get('connections',[]):
            if not each_child in closedSet:   #to avoid 'C' need not revisit
                cameFrom[each_child] = current_node #this is needed before hand to help totalGD
                past_cost = calculate_totalGD(each_child)
                heuristic_cost = calculate_GD(each_child, goal)
                total_cost = past_cost + heuristic_cost
                q.put( (total_cost, each_child) )
                closedSet.add(each_child)
#test
print(list(q.queue))
print(list(closedSet))

[(291, 'F'), (301, 'RV'), (304, 'T'), (311, 'Z'), (552, 'O')]
['RV', 'T', 'S', 'Z', 'A', 'F', 'O']


**ITERATION 3**  
<pre>
Is q empty? No. So Go on.
Take least cost path from q 
{ <font color="blue"><b>(291, 'F')</b></font>, (301, 'RV'), (304, 'T'), (311, 'Z'), (552, 'O') } : <font color="blue"><b>(291, 'F')</b></font>
Current node: 'F'. Kids: 'B'  
Is 'S' the goal? No. So Go on.

                            closedSet : ( 'O', 'RV', 'S', 'A', 'Z', 'T', 'F' )
                            
**'F' to 'B':**          
            past cost from 'A' to 'B' : 138+41+112
       heuristic cost from 'B' to 'U' : 33  
                                total : **324**                                  
                            closedSet : ( 'O', 'RV', 'S', 'A', 'Z', 'T', 'F', 'B' )      
                            
                            
**Result: q = { (301, 'RV'), (304, 'T'), (311, 'Z'), (552, 'O'), **<font color="blue"><b>(324, 'B') }</b></font>
</pre>
<hr>

In [7]:
if not q.empty():
    
    current_cost, current_node = q.get()   
    
    if not (current_node == goal):
        closedSet.add(current_node)
        for each_child in romania_location_map.get(current_node,[]).get('connections',[]):
            if not each_child in closedSet:   #to avoid 'C' need not revisit
                cameFrom[each_child] = current_node #this is needed before hand to help totalGD
                past_cost = calculate_totalGD(each_child)
                heuristic_cost = calculate_GD(each_child, goal)
                total_cost = past_cost + heuristic_cost
                q.put( (total_cost, each_child) )
                closedSet.add(each_child)
#test
print(list(q.queue))
print(list(closedSet))

[(301, 'RV'), (311, 'Z'), (304, 'T'), (552, 'O'), (324, 'B')]
['RV', 'T', 'S', 'Z', 'A', 'B', 'F', 'O']


**ITERATION 4**  
<pre>
Is q empty? No. So Go on.
Take least cost path from q 
{ <font color="blue"><b>(301, 'RV')</b></font>, (311, 'Z'), (304, 'T'), (552, 'O'), (324, 'B') } : <font color="blue"><b>(301, 'RV')</b></font>
Current node: 'RV'. Kids: 'C','P'  
Is 'RV' the goal? No. So Go on.

                            closedSet : ( 'RV', 'Z', 'T', 'A', 'S', 'F', 'B', 'O' )
                            
**'RV' to 'C':**          
            past cost from 'A' to 'C' : 138+49+60
       heuristic cost from 'C' to 'U' : 143  
                                total : **390**                                  
                            closedSet : ( 'RV', 'Z', 'T', 'A', 'S', 'F', 'B', 'O', 'C' ) 
**'RV' to 'P':**          
            past cost from 'A' to 'P' : 138+49+29
       heuristic cost from 'P' to 'U' : 87  
                                total : **303**                                  
                            closedSet : ( 'RV', 'Z', 'T', 'A', 'S', 'F', 'B', 'O', 'C', 'P' ) 
                            
                            
**Result: q = { (311, 'Z'), (304, 'T'), (552, 'O'), (324, 'B'), **<font color="blue"><b>(390, 'C'), (303,'P') }</b></font>
</pre>
<hr>

In [8]:
if not q.empty():
    
    current_cost, current_node = q.get()   
    
    if not (current_node == goal):
        closedSet.add(current_node)
        for each_child in romania_location_map.get(current_node,[]).get('connections',[]):
            if not each_child in closedSet:   #to avoid 'C' need not revisit
                cameFrom[each_child] = current_node #this is needed before hand to help totalGD
                past_cost = calculate_totalGD(each_child)
                heuristic_cost = calculate_GD(each_child, goal)
                total_cost = past_cost + heuristic_cost
                q.put( (total_cost, each_child) )
                closedSet.add(each_child)
#test
print(list(q.queue))
print(list(closedSet))

[(303, 'P'), (311, 'Z'), (304, 'T'), (552, 'O'), (390, 'C'), (324, 'B')]
['RV', 'T', 'P', 'C', 'S', 'Z', 'A', 'B', 'F', 'O']


**ITERATION 5**  
<pre>
Is q empty? No. So Go on.
Take least cost path from q 
{ <font color="blue"><b>(303, 'P')</b></font>, (311, 'Z'), (304, 'T'), (552, 'O'), (390, 'C'), (324, 'B') } : <font color="blue"><b>(303, 'P')</b></font>
Current node: 'P'. Kids: 'B' which is already visited, so do nothing 

                            closedSet : ( 'A', 'B', 'F', 'O', 'P', 'S', 'T', 'Z', 'RV', 'C' )
                                                       
                            
**Result: q = { (311, 'Z'), (304, 'T'), (552, 'O'), (390, 'C'), (324, 'B') }**
</pre>
<hr>

In [9]:
if not q.empty():
    
    current_cost, current_node = q.get()   
    
    if not (current_node == goal):
        closedSet.add(current_node)
        for each_child in romania_location_map.get(current_node,[]).get('connections',[]):
            if not each_child in closedSet:   #to avoid 'C' need not revisit
                cameFrom[each_child] = current_node #this is needed before hand to help totalGD
                past_cost = calculate_totalGD(each_child)
                heuristic_cost = calculate_GD(each_child, goal)
                total_cost = past_cost + heuristic_cost
                q.put( (total_cost, each_child) )
                closedSet.add(each_child)
#test
print(list(q.queue))
print(list(closedSet))
print(cameFrom)

[(304, 'T'), (311, 'Z'), (324, 'B'), (552, 'O'), (390, 'C')]
['RV', 'T', 'P', 'C', 'S', 'Z', 'A', 'B', 'F', 'O']
{'A': None, 'S': 'A', 'T': 'A', 'Z': 'A', 'F': 'S', 'RV': 'S', 'O': 'S', 'B': 'F', 'C': 'RV', 'P': 'RV'}


**ITERATION 6**  
<pre>
Is q empty? No. So Go on.
Take least cost path from q 
{ <font color="blue"><b>(304, 'T')</b></font>, (311, 'Z'), (324, 'B'), (552, 'O'), (390, 'C') } : <font color="blue"><b>(304, 'T')</b></font>
Current node: 'T'. Kids: 'LU'  
Is 'T' the goal? No. So Go on.

                            closedSet : ( 'P', 'O', 'Z', 'F', 'B', 'C', 'T', 'RV', 'S', 'A' )
                            
**'T' to 'LU':**          
            past cost from 'A' to 'LU' : 30+33
       heuristic cost from 'LU' to 'U' : 240  
                                total : **303**                                  
                            closedSet : ( 'P', 'O', 'Z', 'F', 'B', 'C', 'T', 'RV', 'S', 'A', 'LU' ) 
                            
                            
**Result: q = {  (311, 'Z'), (324, 'B'), (552, 'O'), (390, 'C'), **<font color="blue"><b>(303, 'LU') }</b></font>
</pre>
<hr>

In [10]:
if not q.empty():
    
    current_cost, current_node = q.get()   
    
    if not (current_node == goal):
        closedSet.add(current_node)
        for each_child in romania_location_map.get(current_node,[]).get('connections',[]):
            if not each_child in closedSet:   #to avoid 'C' need not revisit
                cameFrom[each_child] = current_node #this is needed before hand to help totalGD
                past_cost = calculate_totalGD(each_child)
                heuristic_cost = calculate_GD(each_child, goal)
                total_cost = past_cost + heuristic_cost
                q.put( (total_cost, each_child) )
                closedSet.add(each_child)
#test
print(list(q.queue))
print(list(closedSet))

[(303, 'LU'), (311, 'Z'), (324, 'B'), (552, 'O'), (390, 'C')]
['RV', 'LU', 'T', 'P', 'C', 'S', 'Z', 'A', 'B', 'F', 'O']


**ITERATION 7**  
<pre>
Is q empty? No. So Go on.
Take least cost path from q 
{ <font color="blue"><b>(303, 'LU')</b></font>, (311, 'Z'), (324, 'B'), (552, 'O'), (390, 'C') } : <font color="blue"><b>(303, 'LU')</b></font>
Current node: 'LU'. Kids: 'M'  
Is 'LU' the goal? No. So Go on.

                            closedSet : ( 'P', 'O', 'LU', 'Z', 'F', 'B', 'C', 'T', 'RV', 'S', 'A' )
                            
**'LU' to 'M':**          
            past cost from 'A' to 'M' : 30+33+58
       heuristic cost from 'LU' to 'U' : 210  
                                total : **331**                                  
                            closedSet : ( 'P', 'O', 'LU', 'Z', 'F', 'B', 'C', 'T', 'RV', 'S', 'A', 'M' ) 
                            
                            
**Result: q = {  (311, 'Z'), (324, 'B'), (552, 'O'), (390, 'C'), **<font color="blue"><b>(331, 'M') }</b></font>
</pre>
<hr>

In [11]:
if not q.empty():
    
    current_cost, current_node = q.get()   
    
    if not (current_node == goal):
        closedSet.add(current_node)
        for each_child in romania_location_map.get(current_node,[]).get('connections',[]):
            if not each_child in closedSet:   #to avoid 'C' need not revisit
                cameFrom[each_child] = current_node #this is needed before hand to help totalGD
                past_cost = calculate_totalGD(each_child)
                heuristic_cost = calculate_GD(each_child, goal)
                total_cost = past_cost + heuristic_cost
                q.put( (total_cost, each_child) )
                closedSet.add(each_child)
#test
print(list(q.queue))
print(list(closedSet))

[(311, 'Z'), (331, 'M'), (324, 'B'), (552, 'O'), (390, 'C')]
['RV', 'LU', 'T', 'P', 'C', 'S', 'M', 'Z', 'A', 'B', 'F', 'O']


**ITERATION 8**  
<pre>
Is q empty? No. So Go on.
Take least cost path from q 
{ <font color="blue"><b>(311, 'Z')</b></font>, (331, 'M'), (324, 'B'), (552, 'O'), (390, 'C') } : <font color="blue"><b>(311, 'Z')</b></font>
Current node: 'Z'. Kids: 'O'  which is already visited so no action
Is 'LU' the goal? No. So Go on.

                            closedSet : ( 'P', 'O', 'LU', 'M', 'Z', 'F', 'B', 'C', 'T', 'RV', 'S', 'A' )                                                   
**Result: q = {  (331, 'M'), (324, 'B'), (552, 'O'), (390, 'C'), **}
</pre>
<hr>

In [12]:
if not q.empty():
    
    current_cost, current_node = q.get()   
    
    if not (current_node == goal):
        closedSet.add(current_node)
        for each_child in romania_location_map.get(current_node,[]).get('connections',[]):
            if not each_child in closedSet:   #to avoid 'C' need not revisit
                cameFrom[each_child] = current_node #this is needed before hand to help totalGD
                past_cost = calculate_totalGD(each_child)
                heuristic_cost = calculate_GD(each_child, goal)
                total_cost = past_cost + heuristic_cost
                q.put( (total_cost, each_child) )
                closedSet.add(each_child)
#test
print(list(q.queue))
print(list(closedSet))
print(cameFrom)

[(324, 'B'), (331, 'M'), (390, 'C'), (552, 'O')]
['RV', 'LU', 'T', 'P', 'C', 'S', 'M', 'Z', 'A', 'B', 'F', 'O']
{'A': None, 'S': 'A', 'T': 'A', 'Z': 'A', 'F': 'S', 'RV': 'S', 'O': 'S', 'B': 'F', 'C': 'RV', 'P': 'RV', 'LU': 'T', 'M': 'LU'}


**ITERATION 9**  
<pre>
Is q empty? No. So Go on.
Take least cost path from q 
{ <font color="blue"><b>(324, 'B')</b></font>, (331, 'M'), (390, 'C'), (552, 'O') } : <font color="blue"><b>(324, 'B')</b></font>
Current node: 'B'. Kids: 'U'  
Is 'B' the goal? No. So Go on.

                            closedSet : ( 'P', 'O', 'LU', 'M', 'Z', 'F', 'B', 'C', 'T', 'RV', 'S', 'A' )
                            
**'B' to 'U':**          
            past cost from 'A' to 'U' : 138+41+112+33  NOTE! SINCE B PARENT IS F, WE CALCULATE VIA THAT PATH :(
       heuristic cost from 'U' to 'U' : 0  
                                total : **324**                                  
                            closedSet : ( 'P', 'O', 'LU', 'M', 'Z', 'F', 'B', 'C', 'T', 'RV', 'S', 'A', 'U' ) 
                            
                            
**Result: q = {  (331, 'M'), (390, 'C'), (552, 'O'), **<font color="blue"><b>(324, 'U') }</b></font>
</pre>
<hr>

In [13]:
if not q.empty():
    
    current_cost, current_node = q.get()   
    
    if not (current_node == goal):
        closedSet.add(current_node)
        for each_child in romania_location_map.get(current_node,[]).get('connections',[]):
            if not each_child in closedSet:   #to avoid 'C' need not revisit
                cameFrom[each_child] = current_node #this is needed before hand to help totalGD
                past_cost = calculate_totalGD(each_child)
                heuristic_cost = calculate_GD(each_child, goal)
                total_cost = past_cost + heuristic_cost
                q.put( (total_cost, each_child) )
                closedSet.add(each_child)
#test
print(list(q.queue))
print(list(closedSet))
print(cameFrom)

[(324, 'U'), (331, 'M'), (390, 'C'), (552, 'O'), (392, 'G')]
['RV', 'LU', 'T', 'P', 'C', 'U', 'S', 'M', 'G', 'Z', 'A', 'B', 'F', 'O']
{'A': None, 'S': 'A', 'T': 'A', 'Z': 'A', 'F': 'S', 'RV': 'S', 'O': 'S', 'B': 'F', 'C': 'RV', 'P': 'RV', 'LU': 'T', 'M': 'LU', 'G': 'B', 'U': 'B'}


**ITERATION 10**  
<pre>
Is q empty? No. So Go on.
Take least cost path from q 
{ <font color="blue"><b>(324, 'U'))</b></font>, (331, 'M'), (390, 'C'), (552, 'O'), (392, 'G') } : <font color="blue"><b>(324, 'B')</b></font>
Current node: 'U'. Kids: 'V'  
Is 'U' the goal? YES. Return. 

current_cost: 324
current_node: 'U'
current_path: ( 'A', 'S', 'F', 'B', 'U')

</pre>
<hr>

In [14]:
if not q.empty():
    
    current_cost, current_node = q.get()   
    
    if (current_node == goal):
        print('Done')     
    else:
        closedSet.add(current_node)
        for each_child in romania_location_map.get(current_node,[]).get('connections',[]):
            if not each_child in closedSet:   #to avoid 'C' need not revisit
                cameFrom[each_child] = current_node
                past_cost = calculate_totalGD(each_child)
                heuristic_cost = calculate_GD(each_child, goal)
                #print('past cost: {} heuristic cost: {}'.format(past_cost,heuristic_cost))
                total_cost = past_cost + heuristic_cost
                q.put( (total_cost, each_child) )
                closedSet.add(each_child)   
#test
print(list(q.queue))
#print(list(closedSet))
print(cameFrom)
print(calculate_path(current_node))

Done
[(331, 'M'), (392, 'G'), (390, 'C'), (552, 'O')]
{'A': None, 'S': 'A', 'T': 'A', 'Z': 'A', 'F': 'S', 'RV': 'S', 'O': 'S', 'B': 'F', 'C': 'RV', 'P': 'RV', 'LU': 'T', 'M': 'LU', 'G': 'B', 'U': 'B'}
['A', 'S', 'F', 'B', 'U']


If some how, the parent of 'B' was 'P' instead of 'F', we could have had better path. So there is still "something" in our algo to be optimized. 

In Iteration 5, when current_node is 'P' and kid was 'B' we simply did nothing. Instead, if we had replaced parent of 'B' from 'F' to 'P', our path would have changed later for better. Isnt it? But what could be justification to do that?

Recall in Iteration 5. cameFrom['B'] is already 'F'. We now have 'P'. Which could be better parent? (Eh?!)

**We will choose better parent which has least g('B') from paths via 'F' and via 'P'.**

<pre>
```cameFrom``` during Iteration 5 which we could use to calculate:  
{'A': None, 'S': 'A', 'T': 'A', 'Z': 'A', 'F': 'S', 'RV': 'S', 'O': 'S', 'B': 'F', 'C': 'RV', 'P': 'RV'}

1) path to 'B' via 'F' : ('B' -> 'F' -> 'S' -> 'A')  costs 112+41+138 = 291  
2) path to 'B' via 'P' : ('B' -> 'P') + ('P' -> 'RV' -> 'S' -> 'A') costs (67) + (29+49+138) = 283
</pre>

Note we calculate ('B' -> 'P') separetely because this is not yet in cameFrom (which is what we are going to decide about). So we cannot use 'calculate_totalGD' which uses 'cameFrom ' list.    
<br><font color='red'>Re read and make sure you understand above explanation, as this is very vital for optimization we would do next</font>

To differentiate 2 calculations above, we will call one has g('B') or **actual g score**. The other one as t_g('B') or **tentative g score**

Thus  
<pre>
1) g('B') = calculate_totalGD('B')  
2) t_g('B') = calculate_GD('P','B') + calculate_totalGD('P')  

if t_g('B') is minimal of above two, then update as cameFrom['B'] = 'P' 
</pre>

As we saw above, ```t_g('B') < g('B')```, that is 283 < 291. so we should update ```cameFrom['B'] = 'P'```

Let us try to insert this change in the code and see what happens.  
Remember 'B' was child here, and 'P' and 'F' were potential parents (P was current node and F was existing in cameFrom as current parent of 'B')

In [15]:
from queue import PriorityQueue
import geopy.distance

def calculate_GD(start, end):

    (x0,y0) = graph.node[start]['pos']
    (x1,y1) = graph.node[end]['pos']
    return floor(geopy.distance.geodesic((y0,x0), (y1,x1)).miles)

def calculate_totalGD(node):
    cost = 0
    if cameFrom.get(node) is None: return 0
    while True:   #when current_node is not None..        
        cost += calculate_GD(cameFrom.get(node),node)
        node = cameFrom.get(node)
        if cameFrom.get(node) is None: break
    return cost

def calculate_path(node):
    path = [node]
    while node is not None:   #when current_node is not None..        
        node = cameFrom.get(node)
        path.append(node)
    return [x for x in reversed(path[:-1])]

cameFrom = {}           
cameFrom[start] = None 

def run_AStar(start, goal):

    # Initialization
    q = PriorityQueue()
    q.put((0,start))
    closedSet = set()  # set() would be better than a list?
    count = 0

    
    # Main loop
    while not q.empty():
        
        
        current_cost, current_node = q.get()   

        if (current_node == goal):
            break      
        else:
            closedSet.add(current_node)
            for each_child in romania_location_map.get(current_node,[]).get('connections',[]):
                                
                if each_child in closedSet:   
                    # if current node or existing parent of visited child, who is better parent?
                    past_cost = calculate_totalGD(each_child)         # this is gScore       
                    t_gScore =  calculate_GD(current_node,each_child) + calculate_totalGD(current_node)
                    #print('T score: ',t_gScore)
                    if (t_gScore < past_cost):
                        cameFrom[each_child] = current_node
                        
                if not each_child in closedSet:   #to avoid 'C' need not revisit
                    cameFrom[each_child] = current_node
                    past_cost = calculate_totalGD(each_child)
                    heuristic_cost = calculate_GD(each_child, goal)
                    #print('past cost: {} heuristic cost: {}'.format(past_cost,heuristic_cost))
                    total_cost = past_cost + heuristic_cost
                    q.put( (total_cost, each_child) )
                    closedSet.add(each_child)   
                                        
        count += 1  
        #print('Iter: {} Node: {} Q:{}'.format(count,current_node,list(q.queue)))
                    
    best_path = calculate_path(current_node)
    return (current_cost, best_path)

start = 'A'
goal = 'LA'
(cost, path) = run_AStar(start,goal)
print(cost, path)
print(cameFrom)
show_map(graph,start=start,goal=goal,path=path,SLDTarget=goal)

494 ['A', 'S', 'RV', 'P', 'B', 'U', 'V', 'LA']
{'A': None, 'S': 'A', 'T': 'A', 'Z': 'A', 'O': 'Z', 'F': 'S', 'RV': 'S', 'B': 'P', 'LU': 'T', 'M': 'LU', 'C': 'D', 'P': 'RV', 'D': 'M', 'G': 'B', 'U': 'B', 'H': 'U', 'V': 'U', 'LA': 'V'}
